#  IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as ts
import cv2
import glob
# Keras Imports

from keras.models import Sequential,Model,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,Input,Activation,ZeroPadding2D,Add,MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import keras.backend as K
from keras.initializers import glorot_uniform


# Sci-kit learn imports
from sklearn.metrics import roc_auc_score,r2_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle



Using TensorFlow backend.


#  LOCATION OF THE DATA

In [2]:
Uninfected = glob.glob(r'C:\Users\G.VENKATARAMANA\DataSets\cell-images-for-detecting-malaria\cell_images\cell_images\Parasitized\*.png')
Infected = glob.glob(r'C:\Users\G.VENKATARAMANA\DataSets\cell-images-for-detecting-malaria\cell_images\cell_images\Parasitized\*.png')


# READING THE IMAGES FROM THE DATASET

In [3]:
data = []
labels = []

for i in Infected:
    img = cv2.imread(i)
    resize_img =cv2.resize(img,(120,120))
    data.append(np.array(resize_img))
    labels.append(0)

for i in Uninfected:
    img = cv2.imread(i)
    resize_img = cv2.resize(img,(120,120))
    data.append(np.array(resize_img))
    labels.append(1)


# COMBINING ALL THE IMAGES AND FORMING THE ARRAY

In [4]:
img_arr  = np.array(data)
label =np.array(labels)
img_arr.shape , label.shape

((27558, 120, 120, 3), (27558,))

# SPLITTING THE DATA

In [5]:
img_arr,label = shuffle(img_arr, label,random_state = 0)

X_train,X_test, y_train,y_test = train_test_split(img_arr ,label,test_size = 0.2,random_state = 0)

In [6]:
 type(y_train)

numpy.ndarray

# CNN


In [7]:
classifier = Sequential()

#Adding 1st Convolution and Pooling Layer
classifier.add(Conv2D(32,kernel_size=(3,3),input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

#Adding 2nd Convolution and Pooling Layer
classifier.add(Conv2D(32,kernel_size=(3,3),activation = 'relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

#Adding 3rd Convolution and Pooling Layer
classifier.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

#Adding 4th Convolution and Pooling Layer
classifier.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

#Flattening
classifier.add(Flatten())

classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid')) 

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


# IMAGE SEGMENTATION

In [8]:
parasite_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
uninfected_datagen = ImageDataGenerator(rescale=1./255)
parasite_data = parasite_datagen.flow_from_directory(r'C:\Users\G.VENKATARAMANA\DataSets\cell-images-for-detecting-malaria\cell_images\cell_images',
                                                     target_size=(64,64),
                                                     batch_size=32,
                                                     class_mode = 'binary')
uninfected_data = uninfected_datagen.flow_from_directory(r'C:\Users\G.VENKATARAMANA\DataSets\cell-images-for-detecting-malaria\cell_images\cell_images',
                                                        target_size=(64,64),
                                                        batch_size=32,
                                                        class_mode = 'binary')

Found 27558 images belonging to 2 classes.
Found 27558 images belonging to 2 classes.


# FITTING THE DATA AND FEEDING TO THE NEURAL NETWORK

In [9]:
accuracies = classifier.fit_generator(parasite_data,
                         steps_per_epoch = 100,
                         epochs = 40,
                         validation_data = uninfected_data,
                         validation_steps = 50)

Epoch 1/40
100/100 [==============================] - 79s 787ms/step - loss: 0.6695 - accuracy: 0.5766 - val_loss: 0.5898 - val_accuracy: 0.7281
Epoch 2/40
100/100 [==============================] - 58s 579ms/step - loss: 0.4879 - accuracy: 0.7844 - val_loss: 0.1721 - val_accuracy: 0.9075
Epoch 3/40
100/100 [==============================] - 52s 518ms/step - loss: 0.2504 - accuracy: 0.9166 - val_loss: 0.1384 - val_accuracy: 0.9406
Epoch 4/40
100/100 [==============================] - 48s 477ms/step - loss: 0.2172 - accuracy: 0.9253 - val_loss: 0.2190 - val_accuracy: 0.9500
Epoch 5/40
100/100 [==============================] - 44s 436ms/step - loss: 0.2159 - accuracy: 0.9353 - val_loss: 0.0888 - val_accuracy: 0.9519
Epoch 6/40
100/100 [==============================] - 41s 405ms/step - loss: 0.1864 - accuracy: 0.9403 - val_loss: 0.1486 - val_accuracy: 0.9406
Epoch 7/40
100/100 [==============================] - 37s 374ms/step - loss: 0.1769 - accuracy: 0.9438 - val_loss: 0.1111 - val_ac